In [ ]:
from exploringqec import *
from qiskit import *
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator



In this notebook, we showcase the basic idea in 3 primary examples. The examples here are:

1. The quantum repetition code 

2. The 9-qubit Shor code

3. Steane's code


## 1. The quantum repetition code


The simplest quantum error correcting code is the quantum repetition code. A quantum repetition code consisting of $n$ data qubits can reliably detect and correct up to $\lfloor (n-1)/2 \rfloor$ bit-flip *or* phase flip errors. That is, the quantum repetition code is a very basic error correcting code that can only handle one type of possible error. To define a quantum repetition code, we specify the size of the code and the type of error we wish to correct. We demonstrate this with $n = 3$ and protect against bit-flip errors. 

In the situation of the $n = 3$ repetition code, the initialization circuit maintains the initial state $\ket{000}$. That is, it simply maps $\ket{000}$ to the ground state $\ket{000}$. Later, we will see examples of codes that have more elaborate initialization circuits  (i.e. that map $\ket{ 0}_{n} $ to a non-trivial ground state).

In [3]:
rep_code_ = quantum_rep_code(3, 'X')
Statevector(rep_code_).draw('Latex')


<IPython.core.display.Latex object>

Using our simple noise model, we introduce noise to the repetition code. We set the parameters so that there is a 5% chance that each of our data qubits may undergo a random Pauli X operation (i.e. bit flip error). For instructional purposes, our noise nodel function has an optional ```print_option``` parameter: when set to ```True```, the function will print out which qubit(s) have gone through error (and which specific error has happened). Of course, in reality, this knowledge will not be available to us, and this print option is strictly for simulation purposes.

In [4]:
rep_code_with_noise = noise_model(0.05,0, rep_code_, print_option=True)

The $3$-qubit repetition code works with $2$ syndrome qubits, the way in which the syndrome qubits are coupled with the data qubits is described by a table. Each key of the table consists of the syndrome qubit index, and the value of the table at a given key corresponds to the indices of the data qubits with which the syndrome qubit will be entangled. 

Upon measuring the syndrome qubits, we will apply a Pauli X operation to a specific qubit, given by a lookup table ```rep_code_decode_table```. The keys of ```rep_code_decode_table``` correspond to a syndrome measurement result, and the values
correspond to the index of data qubit to which we will apply a Pauli X operation (the value of $-1$ indicates to us that no Pauli X should be applied at all).

In [ ]:
rep_code_table = { 0: [0,1], 1: [1,2] }
rep_code_decode_table = {'00':-1 , '01': 0, '11': 1, '10': 2}


In [6]:
syndrome_measure(rep_code_with_noise, rep_code_table, 'Z') 

job = AerSimulator().run(rep_code_with_noise, shots=1, memory=True)   
result = job.result()
memory = result.get_memory(rep_code_with_noise)
target_qubit = rep_code_decode_table[memory[0]]

if target_qubit != -1:
    rep_code_with_noise.x(target_qubit)

In [7]:
result = measure_data(rep_code_with_noise, [rep_code_with_noise.qubits[idx] for idx in range(3)])
print(result)

000


## 2. The 9-qubit Shor code

Now we examine a more elaborate quantum error correcting code. The Shor code consists of $9$ physical data qubits, and can reliably correct up to one X-error (bit-flip error) and one Z-error (phase-flip error)

In [8]:
shor_code_ = shor_code() 


The Shor code circuit takes the initial state $\ket{000000000}$ to the following ground state:

In [9]:

statevector = Statevector(shor_code_)
statevector.draw('Latex')

<IPython.core.display.Latex object>

In [14]:
shor_code_with_noise = noise_model(0.05, 0.05, shor_code_, print_option=True)

Z applied to <Qubit register=(9, "q"), index=5>


Unlike the repetition code, Shor's code can protect against both X-errors and Z-errors simutaneously. In fact, X-errors and Z-errors can be corrected independently from each other. Similar to the code above, for each type of error, we define two kinds of tables: one table describes indices of syndrome qubits and corresponding data qubits with which the syndrome qubit is entangled, and the other table describes indices of data qubits to operate on given a certain syndrome measurement result.

In [ ]:
Z_shor_syndrome_table = {0: [0,1,2,3,4,5] , 1: [3,4,5,6,7,8]}
# Shor's code consists of 2 syndrome ancillas, 
# overlapping in a single block/"plaquette" consisting of 3 qubits

# syndrome outcomes, assuming a single error:
# (0,0) : even parity for all blocks
# (1,0) : odd parity in first block 
# (0,1) : odd parity in third block
# (1,1) : odd parity in second block
Z_shor_decode_table = { '00': -1,  '10' : 0, '01': 6, '11': 3}

In [ ]:
syndrome_measure( shor_code_with_noise ,   Z_shor_syndrome_table, 'X')

job = AerSimulator().run(shor_code_with_noise, shots=1, memory=True)   
result = job.result()
memory = result.get_memory(shor_code_with_noise)
target_qubit = Z_shor_decode_table[memory[0][::-1]]
# print( memory[0][::-1] )
# print(target_qubit)
if target_qubit != -1:
    shor_code_with_noise.z(shor_code_with_noise.qubits[target_qubit])

In [ ]:
X_shor_syndrome_table = {0: [0,1] , 1: [1,2], 2: [3,4], 3: [4,5], 4:[6,7], 5:[7,8]}
X_shor_decode_table = {'000000': -1, '100000': 0, '110000':1, '010000': 2, '001000': 3, '001100':4, '000100':5, '000010': 6, '000011': 7, '000001': 8}
syndrome_measure( shor_code_with_noise ,  X_shor_syndrome_table, 'Z')
job = AerSimulator().run(shor_code_with_noise, shots=1, memory=True)   
result = job.result()
memory = result.get_memory(shor_code_with_noise)[0][::-1].replace(' ', '')[2:]
target_qubit = X_shor_decode_table[memory]
# print(memory)
# print(target_qubit)
if target_qubit != -1:
    shor_code_with_noise.x(target_qubit)
    
    

In the final decoding step, we wish to examine if the state of the data qubits has successfully been maintained in the correct ground state after going through some potential noise. To do this, we simply apply the inverse of the initialization circuit which maps $\ket{000000000}$ to the ground state of the Shor code. If in the end, measuring the the data qubits in the computational basis results in $000000000$, then we are successful.

In [18]:
shor_code_with_noise.compose(shor_code().inverse(), inplace=True )

result  = measure_data(shor_code_with_noise, [shor_code_with_noise.qubits[idx] for idx in range(9)] )
print(result) 

000000000


## 2. Steane's code

Steane's code is another quantum error correcting code design that also protects against single X and single Z errors independently from each other. 

In [20]:
steane_code_ = steane_code()

The groundstate of Steane's code is depicted below. Similar to Steane's code, the initialization circuit maps $\ket{0}_{n}$ to the following groundstate. 

In [21]:
statevector = Statevector(steane_code_)
statevector.draw('Latex')

<IPython.core.display.Latex object>

Again, we introduce noise to the circuit with prescribed probabilities, using our noise model. Then, using predefined static syndrome/decoding tables, we attempt to maintain the correct state and decode back to the initial state $\ket{0}_{n}$ (and measure our system with respect to the computational basis to $00 \cdots 0$)

In [81]:
steane_code_with_noise = noise_model( 0.05,0.05, steane_code_ , print_option=True )

X applied to <Qubit register=(7, "q"), index=4>
Z applied to <Qubit register=(7, "q"), index=5>


In [ ]:
steane_syndrome_table = { 0 : [0,2,4,6], 1: [1,2,5,6] , 2: [3,4,5,6] }
steane_decoding_table = { '001': 0, '010': 1, '100': 3, '011' : 2, '101': 4, '110': 5, '111': 6 }

In [ ]:
syndrome_measure( steane_code_with_noise ,   steane_syndrome_table, 'X')

job = AerSimulator().run(steane_code_with_noise, shots=1, memory=True)   
result = job.result()
memory = result.get_memory(steane_code_with_noise)
target_idx = int(memory[0][-3:],2)-1
if target_idx != -1:
    steane_code_with_noise.z( target_idx )

In [ ]:
syndrome_measure( steane_code_with_noise ,   steane_syndrome_table, 'Z')
job = AerSimulator().run(steane_code_with_noise, shots=1, memory=True)   
result = job.result()
memory = result.get_memory(steane_code_with_noise)
target_idx = int(memory[0][:3],2) - 1
if target_idx != -1:
    steane_code_with_noise.x( target_idx )

In [88]:
steane_code_with_noise.compose(steane_code().inverse(), inplace=True )

result  = measure_data(steane_code_with_noise, [steane_code_with_noise.qubits[idx] for idx in range(7)] )
print(result) 

0000000


 Steane's code only requires 7 qubits instead of the 9 qubits that Shor's code requires. Furthermore, unlike Shor's code, syndrome measurements for both X and Z errors are treated symmetrically. More specifically, note how in the case of Shor's code, separate tables describing the syndrome qubits and decoding procedures are required for X errors and Z errors. In the situation of Steane's code, we only need to define a single pair of syndrome/decoding tables to handle both X and Z errors.



#### Final remarks:

One challenge that one may detect from the above examples, is that the design of syndrome qubit systems and its interaction with the data qubits can be quite delicate. We have already mentioned that the syndrome/decoding table pairs for Shor's code and Steane's code are defined quite differently (where in the Steane situation, we only need to define a single pair of tables). Regardless, we see that in both situations, we use static predefined tables that are specific to some code design -- there is no ready formula or algorithm that produces appropriate tables given a code initialization circuit (for example). In fact, one may also do away with defining a "decoding table" entirely: given a syndrome qubit system, and a resulting measurement of the syndrome qubit system, one may conceive of an algorithm which dynamically determines how one should operate on the code circuit. Indeed, there are code families for which there are such readily available algorithms (for example, surface codes). However, such algorithms are specific to surface codes (or related codes), and there are no known universal algorithms to handle all possible code designs. 


